# IMPORT LIBRARIES AND MODEL 

In [1]:
### IMPORT PACKAGES

print ( "Please wait... \n ----- Model and Libraries are importing---- ")
import cv2
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array


###  LOADING TRAINED MODEL 
model = load_model('inceptionV3-model.h5')

Please wait... 
 ----- Model and Libraries are importing---- 


In [2]:

# define height and width of image which pass through the model 
# Load the Cascade face Classifier

img_width, img_hight = 100, 100
face_cascade = cv2.CascadeClassifier("D:/Uni Project/live mask detection app/haarcascade_frontalface_default.xml")

### make function for image detection

def action1():
  
    # Define parameter for text styling

    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (1, 1)
    class_lable=' '      
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    img_count=1

    # Importe Images from the path 
    # images Converte into grayscale
    # Detect the faces with Cascade detectMultiScale

    image_path = glob.glob("D:/Uni Project/live mask detection app/images/*.jpg")
   
    for image in image_path:
        color_img=cv2.imread(image)
        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_img, 1.1, 6) 

        # (x=axis, y=axis , w=width, h=height) find four point to make rectangle on face 
        # display color images after detecting face area, which is passed throuhg gray images
        # images save in the directory 
        
        for (x, y, w, h) in faces:
            org = (x-10,y-10) 
            print(img_count)
            color_face = color_img[y:y+h,x:x+w] 
            cv2.imwrite('D:/Uni Project/detected images/images from images/input/%dface.jpg'%(img_count),color_face)
            img = load_img('D:/Uni Project/detected images/images from images/input/%dface.jpg'%(img_count), target_size=(img_width,img_hight))
            img_count+=1

            # Convert image into array for the purpose of prediction  
            
            img = img_to_array(img)/255
            img = np.expand_dims(img,axis=0)
            pred_prob = model.predict(img)
            pred=np.argmax(pred_prob)


            # 'without mask' images save in appropriate folder
            # 'with mask' images save in appropriate folder
            
            if pred==0:
                print("User without mask - predic = ",pred_prob[0][0])
                class_lable = "No Mask"
                color = (255, 0, 0)
                cv2.imwrite('D:Uni Project/detected images/images from images/without mask/%dface.jpg'%(img_count),color_face)
                cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
                # Using cv2.putText() method 
                cv2.putText(color_img, class_lable, org, font,fontScale, color, thickness, cv2.LINE_AA)            
            else:
                print('user with mask - prob = ',pred_prob[0][1])
                class_lable = "Mask"
                color = (0, 255, 0)
                cv2.imwrite('D:/Uni Project/detected images/images from images/with mask/%dface.jpg'%(img_count),color_face)
                cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
                # Using cv2.putText() method 
                cv2.putText(color_img, class_lable, org, font,fontScale, color, thickness, cv2.LINE_AA) 
                
# make function for image detection through webcam  

def action2():
    
    # capture images from webcam
    # Define parameter for text styling

    cap = cv2.VideoCapture(0) 
    img_count_full = 0

    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (1, 1)
    class_lable=' '      
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    
    
    while(True):
        img_count_full += 1

        #read image from webcam
        # Convert to grayscale
        # Detect the faces with Cascade detectMultiScale
        
        responce, color_img = cap.read()
        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)

        # (x=axis, y=axis , w=width, h=height) find four point to make rectangle on face 
        # display color images after detecting face area, which is passed throuhg gray images
        # images save in the directory 
        
        img_count = 0
        for (x, y, w, h) in faces:
            org = (x-10,y-10)
            img_count +=1 
            color_face = color_img[y:y+h,x:x+w] 
            cv2.imwrite('D:/Uni Project/detected images/images from live stream/input/%d%dface.jpg'%(img_count_full,img_count),color_face)
            img = load_img('D:/Uni Project/detected images/images from live stream/input/%d%dface.jpg'%(img_count_full,img_count), target_size=(img_width,img_hight))

            img = img_to_array(img)/255
            img = np.expand_dims(img,axis=0)
            pred_prob = model.predict(img)
            #print(pred_prob[0][0].round(2))
            pred=np.argmax(pred_prob)

            # 'without mask' images save in appropriate folder
            # 'with mask' images save in appropriate folder
            
            if pred==0:
                print("User without mask - predic = ",pred_prob[0][0])
                class_lable = "No Mask"
                color = (255, 0, 0)
                cv2.imwrite('D:/Uni Project/detected images/images from live stream/without mask/%d%dface.jpg'%(img_count_full,img_count),color_face)
            else:
                print('user with mask - prob = ',pred_prob[0][1])
                class_lable = "Mask"
                color = (0, 255, 0)
                cv2.imwrite('D:/Uni Project/detected images/images from live stream/with mask/%d%dface.jpg'%(img_count_full,img_count),color_face)

            # Opencv method rectangle() & putText()
            # all text parameter is defined above  
            
            cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(color_img, class_lable, org, font,  
            fontScale, color, thickness, cv2.LINE_AA) 

        # Display the image window 
            
        cv2.imshow('LIVE face mask detection', color_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object
    
    cap.release()
    cv2.destroyAllWindows()
    
    
    
    
    
# make function for image detection through Recorded video
def action3():
    
    # path of video 
    # Define parameter for text styling
    
    cap = cv2.VideoCapture('D:/Uni Project/live mask detection app/videos/my_video.mp4') # for video
    img_count_full = 0
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (1, 1)
    class_lable=' '      
    fontScale = 1 
    color = (255, 0, 0) 
    thickness = 2
    
    ## Start reading images and prediction
    
    while(True):
        img_count_full += 1
        
        # read image from video
        responce, color_img = cap.read()

        #after complete the video it will terminate the reading 
        if responce == False:
            break    

        # Convert to grayscale
        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

        # Detect the faces with Cascade detectMultiScale
        faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)

        # (x=axis, y=axis , w=width, h=height) find four point to make rectangle on face 
        # display color images after detecting face area, which is passed throuhg gray images
        # images save in the directory 
        img_count = 0
        for (x, y, w, h) in faces:
            org = (x-10,y-10)
            img_count += 1 
            color_face = color_img[y:y+h,x:x+w] 
            cv2.imwrite('D:/Uni Project/detected images/images from recorded video/input/%d%dface.jpg'%(img_count_full,img_count),color_face)
            img = load_img('D:/Uni Project/detected images/images from recorded video/input/%d%dface.jpg'%(img_count_full,img_count), target_size=(img_width,img_hight))

            # Rescale images for prediction purpose
            
            img = img_to_array(img)/255
            img = np.expand_dims(img,axis=0)
            pred_prob = model.predict(img)
            pred=np.argmax(pred_prob)

            # 'without mask' images save in appropriate folder
            # 'with mask' images save in appropriate folder
            
            if pred==0:
                print("User without mask - predic = ",pred_prob[0][0])
                class_lable = "No Mask"
                color = (255, 0, 0)
                cv2.imwrite('D:/Uni Project/detected images/images from recorded video/without mask/%d%dface.jpg'%(img_count_full,img_count),color_face)
            else:
                print('user with mask - prob = ',pred_prob[0][1])
                class_lable = "Mask"
                color = (0, 255, 0)
                cv2.imwrite('D:/Uni Project/detected images/images from recorded video/with mask/%d%dface.jpg'%(img_count_full,img_count),color_face)

            # Opencv method rectangle() & putText()
            # all text parameter is defined above  
            
            cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(color_img, class_lable, org, font, fontScale, color, thickness, cv2.LINE_AA) 

        # display image
        cv2.imshow('LIVE face mask detection', color_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object
    cap.release()
    cv2.destroyAllWindows()



In [3]:
    
def processing():
    print("-----------------------")
    print( "  Under Proccessing")   
    print("-----------------------")

def completed():
    print("-----------------------")
    print("job is completed")
    print("-----------------------")    

print(" ------ MODEL HAS BEEN LOADED ------- ")
user_input = input("Please Select anyone option \n | 1 : Image Detection | \n | 2 : webcam detection  | \n | 3 : Recorded Frames |\n" )

if user_input == '1':
    print("| 1 : Image Detection |")
    processing()
    print(action1())
    completed()
elif user_input == '2':
    print("| 2 : webcam detection  |")
    processing()
    print(action2())
    completed()
elif user_input == '3':
    print("| 3 : Recorded Frames | ")
    processing()
    print(action3())
    completed()
else:
    print("Invalid command \n please Rerun the code")

 ------ MODEL HAS BEEN LOADED ------- 
Please Select anyone option 
 | 1 : Image Detection | 
 | 2 : webcam detection  | 
 | 3 : Recorded Frames |
1
| 1 : Image Detection |
-----------------------
  Under Proccessing
-----------------------
1
user with mask - prob =  1.0
2
user with mask - prob =  0.9999975


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Raza Pc\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-35f53aad6e4e>", line 17, in <module>
    print(action1())
  File "<ipython-input-2-bc0840294649>", line 30, in action1
    faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Raza Pc\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py", line 2054, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Raza Pc\.conda\envs\tf\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fi

TypeError: object of type 'NoneType' has no len()